## 优先使用F-string风格的代码内容, 而不是C-style的或者`str.format`形式的

## 直接使用`%`

- 首先先观察下述例子, 下述例子关于一个format的风格`%d`, 代表一个占位符, 也就是被替换的位置:

In [1]:
a = 0b10111011
b = 0xc5f
print('二进制数是%d, 十六进制的是%d' % (a, b))

二进制数是187, 十六进制的是3167


- 以上的format style是C-style的, 很可能报错, 例如你将格式稍微转换一下就会报错:

In [2]:
key = 'my_var'
value = 1.234
# `-10`表示共十位数, 但是在后面加上, `+10`则表示在句首添加缺少的数, 可以`print(list(format))`查看相关的函数
formatted = '%+10s = %.2f' % (key, value)
print(formatted)

    my_var = 1.23


In [3]:
# 但是经过一次转换之后我们会得到:

key = 'my_var'
value = 1.234
reordered_tuple = '%-10s = %.2f' % (value, key)
print(reordered_tuple)

TypeError: must be real number, not str

以上的内容给我们的使用带来很大的不方便之处, 这是由于每次修改一点, 所有的东西都必须手动修改

- 此处可能存在的第二个问题为: 若想要修改内容实际上是不方便阅读的

In [4]:
pantry = [
 ('avocados', 1.25),
 ('bananas', 2.5),
 ('cherries', 15),
]

for i, (item, count) in enumerate(pantry):
    print('#%d: %-10s = %.2f' % (i, item, count))

#0: avocados   = 1.25
#1: bananas    = 2.50
#2: cherries   = 15.00


In [5]:
# 此时加入一点其他的格式上的修改, 例如 count要整数, 首字母大写

for i, (item, count) in enumerate(pantry):
    print('#%d: %-10s = %d' % (
    i + 1,
    item.title(),
    round(count)))

#1: Avocados   = 1
#2: Bananas    = 2
#3: Cherries   = 15


In [6]:
# 此时使用字典的方式进行:
key = 'my_var'
value = 1.234

old_way = '%-10s = %.2f' % (key, value)
new_way = '%(key)-10s = %(value).2f' % {
 'key': key, 'value': value} # Original
reordered = '%(key)-10s = %(value).2f' % {
 'value': value, 'key': key} # Swapped

In [7]:
# 发现其仍然相等, 说明这种方式的可行性
assert old_way == new_way == reordered

- 但是这种方法仍然增加了代码的冗长, 这也是C-style所带来的问题, 为了证明这个真的很冗长, 作者给出了以下的例子:
  - 当然个人认为代码高亮其实能够一定程度上缓解这个问题, 不过我自己本人没有接触到大型项目, 不知道这个是不是这样.

In [8]:
menu = {
 'soup': 'lentil',
 'oyster': 'kumamoto',
 'special': 'schnitzel', }
template = ('Today\'s soup is %(soup)s, '
 'buy one get two %(oyster)s oysters, '
 'and our special entrée is %(special)s.')
formatted = template % menu
print(formatted)

Today's soup is lentil, buy one get two kumamoto oysters, and our special entrée is schnitzel.


## 使用`.format`

- `.format()`本质上是一个*built-in function*, 也就是说这是一个`__format__`方法

In [9]:
a = 1234.5678
formatted = format(a, ',.2f')
print(formatted)

b = 'my string'
# `^`表示居中, `<`表示靠左, `>`表示靠右
formatted = format(b, '<13s')
print('*', formatted, '*')

1,234.57
* my string     *


- 如果要在使用这些方法的时候escape调用这些方法, 直接打印对应的符号的话则需要double一下相关的符号, 例如:

In [10]:
print('%.2f%%' % 12.5)
print('{} replaces {{}}'.format(1.23))

12.50%
1.23 replaces {}


- format中还有一些比较特殊的trick可以使用, 例如使用index来format我们的string, 这样可以解决前述所提到的一些问题:
  - 比如我们只用change后半部分的format的部分就可以无报错地执行我们的代码(*参考前述报错部分的内容*)

In [11]:
formatted = '{1} = {0}'.format(key, value)
print(formatted)

1.234 = my_var


- 为了避免冗长, 该方法仅仅要求我们只使用一种方法进行

In [12]:
name = 'Max'

formatted = '{0} loves food. See {0} cook.'.format(name)
print(formatted)

Max loves food. See Max cook.


- 可读性增强

In [13]:
pantry = [
 ('avocados', 1.25),
 ('bananas', 2.5),
 ('cherries', 15),
]

for i, (item, count) in enumerate(pantry):
    old_style = '#%d: %-10s = %d' % (
      i + 1,
      item.title(),
      round(count))

    new_style = '#{}: {:<10s} = {}'.format(
      i + 1,
      item.title(),
      round(count))
    
    assert old_style == new_style

In [14]:
# 一些更advance的方法, 使代码更简洁, 注意下述注释解释了为何这个代码可以如此写

# such as using combinations of dictionary keys 
# and list indexes in placeholders, and **coercing values** to Unicode and 
# repr strings:

menu = {
    'soup': 'lentil',
    'oyster': 'kumamoto',
    'special': 'schnitzel', }

formatted = 'First letter is {menu[oyster][0]!r}'.format(
    menu=menu)

print(formatted)


First letter is 'k'


- 但是这种代码的可读性貌似仍然有点不行:

In [15]:
old_template = (
    'Today\'s soup is %(soup)s, '
    'buy one get two %(oyster)s oysters, '
    'and our special entrée is %(special)s.')

old_formatted = template % {
    'soup': 'lentil',
    'oyster': 'kumamoto',
    'special': 'schnitzel', }

new_template = (
    'Today\'s soup is {soup}, '
    'buy one get two {oyster} oysters, '
    'and our special entrée is {special}.')

new_formatted = new_template.format(
    soup='lentil',
    oyster='kumamoto',
    special='schnitzel', )
assert old_formatted == new_formatted

## `f-string`风格

- 这种风格简单多了

In [16]:
key = 'my_var'
value = 1.234

formatted = f'{key} = {value}'
print(formatted)

my_var = 1.234


- 如果使用上面的trick呢?
  - 添加 `!r`代码

In [17]:
formatted = f'{key!r:<10} = {value:.2f}'
print(formatted)

'my_var'   = 1.23


- 简单对比几种风格

In [18]:
f_string = f'{key:<10} = {value:.2f}'

c_tuple = '%-10s = %.2f' % (key, value)

str_args = '{:<10} = {:.2f}'.format(key, value)

str_kw = '{key:<10} = {value:.2f}'.format(key=key,
    value=value)

c_dict = '%(key)-10s = %(value).2f' % {'key': key,
    'value': value}

assert c_tuple == c_dict == f_string
assert str_args == str_kw == f_string

In [19]:
for i, (item, count) in enumerate(pantry):
    
    old_style = '#%d: %-10s = %d' % (
        i + 1,
        item.title(),
        round(count))

    new_style = '#{}: {:<10s} = {}'.format(
        i + 1,
        item.title(),
        round(count))
    
    f_string = f'#{i+1}: {item.title():<10s} = {round(count)}'
    
    assert old_style == new_style == f_string

- 可以将这种写法换行书写, 使得可读性更好:

In [20]:
for i, (item, count) in enumerate(pantry):
    print(f'#{i+1}: '
    f'{item.title():<10s} = '
    f'{round(count)}')

#1: Avocados   = 1
#2: Bananas    = 2
#3: Cherries   = 15


- 嵌套大法开启: 可以使用变量作为`f-string`的参数传入, 传入方式如下所示:

In [21]:
places = 3
number = 1.23456

# 使用两个大括号:
print(f'My number is {number:.{places}f}')

My number is 1.235


## 总结

使用以上的例子说明了f-string真的很优雅简洁, 同时简明介绍了几种方式的优缺点

- C-style format strings that use the % operator suffer from a variety of gotchas and verbosity problems.
- The str.format method introduces some useful concepts in its formatting specifiers mini language, but it otherwise repeats the mistakes of C-style format strings and should be avoided.
- F-strings are a new syntax for formatting values into strings that solves the biggest problems with C-style format strings.
- F-strings are succinct yet powerful because they allow for arbitrary Python expressions to be directly embedded within format specifiers.